In [1]:
import numpy as np
import pandas as pd
from comet_ml import Experiment
import xgboost as xgb
import warnings
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score
import numpy as np
from format_data import *

# Entraînement

### Importation et formattage des données

In [2]:
train_data = pd.read_csv("./data/GAN_train.csv", index_col=0)
train_data = format_data(train_data)
train_data.head()

,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,PSL,T200,T500,...,TMQ_WINDBOT_INTERACTION,QREFHT_TREFHT_INTERACTION,TMQ_SQUARE,WIND850_MAGNITUDE_SQUARE,WINDBOT_MAGNITUDE_SQUARE,TREFHT_SQUARE,MONTH_SIN,MONTH_COS,LAPSE_RATE,Label
0,14.278533,-3.173287,1.833835,-7.581432,3.399386,0.006412,103070.359400,103070.359400,214.141617,256.000519,...,118.635566,1.842325,203.876504,13.432702,69.033935,82562.958151,-0.866025,0.500000,0.139530,0
1,44.269070,-3.000973,0.375796,-2.194901,-3.694045,0.017199,101155.218800,101155.218800,219.897949,266.757843,...,190.220828,5.165643,1959.750529,9.147060,18.463556,90207.229732,-0.866025,0.500000,0.156200,0
2,29.716904,5.699519,-4.106040,3.079108,-2.678627,0.012480,101841.164100,101841.164100,215.238892,264.979614,...,121.279737,3.674712,883.094365,49.344087,16.655949,86705.590264,-0.866025,0.500000,0.165802,2
3,36.301456,-0.657589,-1.584453,-0.903698,-2.157597,0.017055,101450.968800,101450.968800,217.602081,267.198273,...,84.916650,5.140841,1317.795740,2.942915,5.471893,90860.960406,-0.866025,-0.500000,0.165321,0
4,29.445889,22.721797,-15.070397,1.895406,-7.071098,0.008437,101568.182752,101137.479833,215.028439,265.348473,...,215.565199,2.459911,867.060350,743.396917,53.592988,85008.390575,-0.500000,-0.866025,0.167733,2


### Division du jeu de données en jeu d'entraînement et de validation

In [3]:
y = train_data["Label"]
X = train_data.drop(columns=["Label"])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

### Calcul des poids à donner à chacune des classes

In [4]:
class_weights = compute_class_weight('balanced', classes=[0, 1, 2], y=y_train)
weights = {0: class_weights[0], 1: class_weights[1], 2: class_weights[2]}

train_dmatrix = xgb.DMatrix(X_train, label=y_train, weight=y_train.map(weights))
val_dmatrix = xgb.DMatrix(X_val, label=y_val)

c:\Users\Utilisateur\miniconda3\envs\kaggle1\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\Utilisateur\miniconda3\envs\kaggle1\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\Utilisateur\miniconda3\envs\kaggle1\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\Utilisateur\miniconda3\envs\kaggle1\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype)

### Entraînement

In [5]:
from xgboost.callback import TrainingCallback

class CustomLearningRateCallback(TrainingCallback):
    def __init__(self):
        super().__init__()

    def before_iteration(self, model, epoch, evals_log):
        if epoch < 50:
            model.set_param('eta', 0.1)
        elif epoch < 150:
            model.set_param('eta', 0.05)
        else:
            model.set_param('eta', 0.01)
        return False

In [6]:
hyperparameter = {
    'objective': 'multi:softprob',
    'num_class': 3,
    'booster': 'gbtree',
    'eval_metric': 'merror',
    'eta': 0.01,
    'max_depth': 8,
    'subsample': 0.8,
    'colsample_bytree': 0.5,
    'alpha': 0.05,
    'seed': 42,
    'num_round': 1000000
}

watchlist = [(train_dmatrix, 'train'), (val_dmatrix, 'validation')]

xgb_model = xgb.train(
hyperparameter, 
train_dmatrix, 
num_boost_round=1000000, 
evals=watchlist, 
early_stopping_rounds=75, 
verbose_eval=True
)

c:\Users\Utilisateur\miniconda3\envs\kaggle1\lib\site-packages\xgboost\core.py:160: UserWarning: [16:09:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "num_round" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-merror:0.05993	validation-merror:0.11856


[1]	train-merror:0.04739	validation-merror:0.09356
[2]	train-merror:0.04819	validation-merror:0.09544
[3]	train-merror:0.04621	validation-merror:0.09107
[4]	train-merror:0.04282	validation-merror:0.08029
[5]	train-merror:0.04010	validation-merror:0.07841
[6]	train-merror:0.04184	validation-merror:0.07919
[7]	train-merror:0.04125	validation-merror:0.08076
[8]	train-merror:0.04077	validation-merror:0.07857
[9]	train-merror:0.04369	validation-merror:0.08091
[10]	train-merror:0.04397	validation-merror:0.07919
[11]	train-merror:0.04514	validation-merror:0.08076
[12]	train-merror:0.04326	validation-merror:0.07841
[13]	train-merror:0.04214	validation-merror:0.07560
[14]	train-merror:0.04344	validation-merror:0.07623
[15]	train-merror:0.04319	validation-merror:0.07591
[16]	train-merror:0.04337	validation-merror:0.07779
[17]	train-merror:0.04283	validation-merror:0.07748
[18]	train-merror:0.04397	validation-merror:0.07716
[19]	train-merror:0.04402	validation-merror:0.07685
[20]	train-merror:0.0

In [12]:
y_pred = xgb_model.predict(val_dmatrix)
y_pred = [np.argmax(value) for value in y_pred]

accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

Validation Accuracy: 99.39%


# Calcul des prédictions sur le jeu de données test

In [13]:
import pandas as pd

# Assuming train_data has been previously loaded
test_data = pd.read_csv("./data/test.csv")

# Format the test_data
test_data = format_data(test_data, is_test=True)

# List of columns from train_data excluding the label
columns_without_label = [col for col in train_data.columns if col != "Label"]

# Check and add missing columns to test_data and set their values to 0
missing_columns = ['SEASON_Spring', 'SEASON_Winter']

for col in missing_columns:
    if col not in test_data.columns:
        test_data[col] = 0

# Reorder the columns of test_data to match the order in train_data
test_data = test_data[columns_without_label]

test_data

,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,PSL,T200,T500,...,TMQ_WIND850_INTERACTION,TMQ_WINDBOT_INTERACTION,QREFHT_TREFHT_INTERACTION,TMQ_SQUARE,WIND850_MAGNITUDE_SQUARE,WINDBOT_MAGNITUDE_SQUARE,TREFHT_SQUARE,MONTH_SIN,MONTH_COS,LAPSE_RATE
0,25.907482,6.662070,-17.510447,-7.432653,-3.936030,0.010624,101532.5391,101532.5391,213.092209,256.032043,...,485.375771,217.895122,3.081385,671.197631,350.998912,70.736668,84123.171146,-0.866025,0.5,0.143133
1,25.907482,6.662070,-17.510447,-7.432653,-3.936030,0.010624,101532.5391,101532.5391,213.092209,256.032043,...,485.375771,217.895122,3.081385,671.197631,350.998912,70.736668,84123.171146,-0.866025,0.5,0.143133
2,27.019733,4.951319,-17.341263,-7.286631,-3.150316,0.010890,101513.0234,101513.0234,213.161011,255.616837,...,487.281212,214.495643,3.157487,730.065995,325.234953,63.019482,84065.558951,-0.866025,0.5,0.141519
3,27.019733,4.951319,-17.341263,-7.286631,-3.150316,0.010890,101513.0234,101513.0234,213.161011,255.616837,...,487.281212,214.495643,3.157487,730.065995,325.234953,63.019482,84065.558951,-0.866025,0.5,0.141519
4,26.516499,5.362008,-17.227922,-7.257047,-2.907396,0.010821,101505.1484,101505.1484,213.188248,255.498810,...,478.439089,207.300143,3.138607,703.124696,325.552442,61.117679,84123.985464,-0.866025,0.5,0.141035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10315,57.411018,3.704696,17.998688,-8.723821,14.131296,0.020041,100295.9375,100295.9375,222.533188,267.842407,...,1054.985123,953.435377,6.022164,3296.025030,337.677536,275.798579,90291.318099,-0.866025,0.5,0.151031
10316,57.277252,5.944778,18.429197,-8.424341,15.523009,0.020222,100210.2891,100210.2891,221.928162,268.028778,...,1109.133280,1011.609908,6.077682,3280.683620,374.975699,311.933342,90325.397350,-0.866025,0.5,0.153669
10317,57.277252,5.944778,18.429197,-8.424341,15.523009,0.020222,100210.2891,100210.2891,221.928162,268.028778,...,1109.133280,1011.609908,6.077682,3280.683620,374.975699,311.933342,90325.397350,-0.866025,0.5,0.153669
10318,54.855862,8.170049,19.165062,-6.791406,16.326492,0.020324,100116.5234,100116.5234,220.904862,267.992645,...,1142.858780,969.999232,6.107221,3009.165558,434.049296,312.677549,90295.041208,-0.866025,0.5,0.156959


In [14]:
dtest = xgb.DMatrix(test_data)
y_test = xgb_model.predict(dtest)
y_test = [np.argmax(value) for value in y_test]

c:\Users\Utilisateur\miniconda3\envs\kaggle1\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\Utilisateur\miniconda3\envs\kaggle1\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\Utilisateur\miniconda3\envs\kaggle1\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\Utilisateur\miniconda3\envs\kaggle1\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype)

In [15]:
labels, counts = np.unique(y_test, return_counts=True)

for label, count in zip(labels, counts):
    print(f"Label {label}: {count} occurrences")

Label 0: 7862 occurrences
Label 1: 622 occurrences
Label 2: 1836 occurrences


In [16]:
df = pd.DataFrame({
    'SNo': range(1, len(y_test) + 1),
    'Label': y_test
})

df.to_csv("xgboost_prev.csv", index=False)